#### **Exercise 1:**

Last week, we started looking at the Titanic data.  Load that data again. For features that you think are of minimal importance and / or have too many NAs to make imputation feasible, drop those features.  For those that seem important, use a `SimpleImputer` to impute the data.

In [28]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
import numpy as np
titanic_data = pd.read_csv("data/titanic.csv")
td_clean = titanic_data.drop('cabin', axis=1)
td_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
td_clean['age'] = td_imputer.fit_transform(td_clean[['age']])
td_clean['fare'] = td_imputer.fit_transform(td_clean[['fare']])
td_imputer = SimpleImputer(strategy='most_frequent')
td_clean['embarked'] = td_imputer.fit_transform(td_clean[['embarked']])[:, 0]
td_clean.isna().sum()





pclass      0
name        0
sex         0
age         0
sibsp       0
parch       0
ticket      0
fare        0
embarked    0
survived    0
dtype: int64

#### **Exercise 2**

Sklearn does not handle strings.  Use an encoder to transform any string columns into numbers. If there are any categorical columns where label encoding won't work, use a one-hot encoder.

In [34]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
string_columns = td_clean['embarked', '']
onehot_encoder = OneHotEncoder(drop='first', sparse_output=False)  # drop='first' to avoid multicollinearity

# Use ColumnTransformer to apply OneHotEncoder to all string columns
column_transformer = ColumnTransformer(transformers=[('onehot', onehot_encoder, string_columns)], remainder='passthrough')

# Apply transformation and return as DataFrame
td_clean_encoded = pd.DataFrame(column_transformer.fit_transform(td_clean), columns=column_transformer.get_feature_names_out())

# Check the transformed dataset
print(td_clean_encoded.head())
print(titanic_data.head())

   remainder__pclass  remainder__name  remainder__sex  remainder__age  \
0                1.0             21.0             0.0         29.0000   
1                1.0             23.0             1.0          0.9167   
2                1.0             24.0             0.0          2.0000   
3                1.0             25.0             1.0         30.0000   
4                1.0             26.0             0.0         25.0000   

   remainder__sibsp  remainder__parch  remainder__ticket  remainder__fare  \
0               0.0               0.0              187.0         211.3375   
1               1.0               2.0               49.0         151.5500   
2               1.0               2.0               49.0         151.5500   
3               1.0               2.0               49.0         151.5500   
4               1.0               2.0               49.0         151.5500   

   remainder__embarked  remainder__survived  
0                  2.0                  1.0  
1     

#### **Exercise 3**

Using 5-fold cross-validation, examine the performance of a LogisticRegression classifier. 

In [36]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
X = td_clean.drop('survived', axis=1)
y = td_clean['survived']
result = cross_val_score(lr, X, y, cv=5)

/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-re

#### **Exercise 4**

Replace the logistic regression classifier with a decision tree classifier.  Which works better?  Why do you think that is?

In [39]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
X = td_clean.drop('survived', axis=1)
y = td_clean['survived']
result = cross_val_score(dt, X, y, cv=5)
result

array([0.4351145 , 0.65267176, 0.08778626, 0.16030534, 0.57471264])

#### **Exercise 5:** 

See if you can get the decision tree classifier to perform better by adjusting your imputation procedure to use a KNNImputer.

#### **Exercise 6:**

Try using a `GridSearchCV` to optimize the DecisionTree algorithm.  What is the best performance you can achieve?